# calculate the density weighted joint PDF of Z, C

In [2]:
import copy
import numpy as np
import matplotlib.pyplot as plt
from counterflow_file import *
from weighted_gaussian_kde import gaussian_kde

In [3]:
# parameters
#models = ['IEM','IEMHYB','EMST','EMSTHYB']
models = ['EMSTHYB',]
params = {}
params['MIX'] = None
params['tres'] = 1.e-2
params['tmix'] = 0.2
params['eqv'] = 1.0
params['Zfvar'] = 0.1
params['dtmix'] = 0.01
params['phif'] = 4.76

csv_name = 'ZCTR.csv'

c_lb = 0
c_ub = 0.3
c_npts = 301

z_lb = 0
z_ub = 0.3
z_npts = 301

# points per step
pts = 10

c_all = np.linspace(c_lb, c_ub, c_npts)
z_all = np.linspace(z_lb, z_ub, z_npts)
v_all = np.zeros((len(c_all),len(z_all)))

In [4]:
for i, model in enumerate(models):
    params['MIX'] = model
    case_name = params2name(params)
    
    data = np.genfromtxt('/'.join([case_name,csv_name]),
                         delimiter=',',
                         names=True)
    
    zc = np.column_stack((data['Z'],data['C']))
    pdf = gaussian_kde(zc.transpose(), weights=data['R'])
    
    for j in range(z_npts//pts):
        z = z_all[ j*pts : (j+1)*pts ]
        for k in range(c_npts//pts):
            c = c_all[ k*pts : (k+1)*pts ]
            
            zm, cm = np.meshgrid(z, c)
            
            v = pdf((np.ravel(zm), np.ravel(cm)))
            v = np.reshape(v, zm.shape)
            
            v_all[k*pts:(k+1)*pts,j*pts:(j+1)*pts] = v
            
    file_name = 'pdf_zc_z-{0:g}-{1:g}_c-{2:g}-{3:g}.csv'.format(
        z_lb, z_ub, c_lb, c_ub)
    np.savetxt('/'.join([case_name, file_name]),v_all)